In [1]:
# Import the utility module
import notebook_utils

# Setup the path
notebook_utils.setup_path()

In [7]:
# Now you can import from src
from src.models import MLP, ResNet, CNN, VisionTransformer
from src.utils import metrics, monitor
import torch
import torch.nn as nn

inputs = torch.randn(32,3,32,32)
targets = torch.randint(0,2,size=(32,),dtype=torch.long)

model_name = 'vit'
if model_name=='mlp':
    model = MLP(input_size=10, output_size=2)
    inputs = torch.randn(32,10)
elif model_name=='cnn':
    model = CNN(in_channels=3,num_classes=2)
elif model_name=='resnet':
    model = ResNet(in_channels=3,num_classes=2)
elif model_name=='vit':
    model= VisionTransformer(in_channels=3,img_size=32)
else:
    raise ValueError(f'model {model_name} does not exist')

criteria = nn.CrossEntropyLoss()

monitor = monitor.NetworkMonitor(model)
monitor.register_hooks()
# model
out = model(inputs)
loss = criteria(out,targets)
loss.backward()

acts = monitor.get_latest_activations()
grads = monitor.get_latest_gradients()

for k,a in acts.items():
    g = grads[k]
    a_flatten = metrics.flatten_activations(a)
    print(f"{k} -> {a.shape} -(flatten)-> {a_flatten.shape}")

In [8]:
model

## Example: Using modules from src

In [ ]:
import torch
from src.models.mlp import MLP

# Create a simple MLP model
model = MLP(
    input_size=784,
    hidden_sizes=[128, 64],
    output_size=10,
    activation='relu'
)

print(model)

In [ ]:
# Generate random input
torch.manual_seed(42)
x = torch.randn(10, 784)

# Run forward pass
with torch.no_grad():
    output = model(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

## Accessing the new Gaussianity metrics

In [ ]:
from src.utils.metrics import measure_gaussianity

# Generate random activations
gaussian_activations = torch.randn(1000, 10)  # Normal distribution
uniform_activations = torch.rand(1000, 10)    # Uniform distribution

# Measure Gaussianity
gaussian_score = measure_gaussianity(gaussian_activations, method="shapiro")
uniform_score = measure_gaussianity(uniform_activations, method="shapiro")

print(f"Gaussianity score for normal distribution: {gaussian_score:.4f}")
print(f"Gaussianity score for uniform distribution: {uniform_score:.4f}")
print(f"Uniform distribution is {uniform_score/gaussian_score:.1f}x less Gaussian")

## Reloading modules after changes

If you make changes to files in the `src` directory while working in the notebook, you can reload the modules to pick up those changes.

In [ ]:
# After making changes to src/utils/metrics.py
import importlib
from src.utils import metrics

# Reload the module
metrics = importlib.reload(metrics)

# Or if using the utility module:
# metrics = notebook_utils.reload_module(metrics)